In [1]:
%time
import pandas as pd
import os
##os.chdir('C:/Users/575774/Documents/project')
os.chdir('C:/Users/nisha/OneDrive/Documents/project')
xls = pd.ExcelFile('Housing_Prices_2005_2023.xlsx')
df = pd.DataFrame()
cities_df = pd.DataFrame()

for sheet in xls.sheet_names:
    sheet_df=pd.read_excel('Housing_Prices_2005_2023.xlsx', sheet_name=sheet)
    province = ['BRITISH_COLUMBIA','ALBERTA','SASKATCHEWAN', 'WINNIPEG', 'ONTARIO','QUEBEC','NOVA_SCOTIA','PRINCE_EDWARD_ISLAND','NEWFOUNDLAND_AND_LABRADOR']
    if sheet in province:
        sheet = 'MANITOBA' if sheet== 'WINNIPEG' else sheet
        sheet_df['Location'] = sheet
        df = pd.concat([df,sheet_df])
    else:
        sheet_df['Location'] = sheet    
        cities_df = pd.concat([cities_df,sheet_df])
df.head


<bound method NDFrame.head of           Date  Composite_HPI_SA  Single_Family_HPI_SA  One_Storey_HPI_SA   
0   2005-01-01             100.0                 100.0              100.0  \
1   2005-02-01             100.6                 100.5              100.8   
2   2005-03-01             101.4                 101.2              102.0   
3   2005-04-01             101.9                 101.8              102.8   
4   2005-05-01             102.6                 102.3              103.6   
..         ...               ...                   ...                ...   
216 2023-01-01             214.7                 215.7              231.6   
217 2023-02-01             214.9                 215.8              231.0   
218 2023-03-01             213.6                 214.7              228.6   
219 2023-04-01             215.0                 215.9              230.2   
220 2023-05-01             216.6                 217.6              230.6   

     Two_Storey_HPI_SA  Townhouse_HPI_SA  Apa

In [2]:
df.columns
df.shape

(1989, 14)

In [3]:
%time
df.drop('Composite_HPI_SA', axis=1,inplace=True)
df.drop('Single_Family_HPI_SA', axis=1,inplace=True)
df.drop('One_Storey_HPI_SA', axis=1,inplace=True)
df.drop('Two_Storey_HPI_SA', axis=1,inplace=True)
df.drop('Townhouse_HPI_SA', axis=1,inplace=True)
df.drop('Apartment_HPI_SA', axis=1,inplace=True)
df.drop('Composite_Benchmark_SA', axis=1,inplace=True)

df.columns


CPU times: total: 0 ns
Wall time: 0 ns


Index(['Date', 'Single_Family_Benchmark_SA', 'One_Storey_Benchmark_SA',
       'Two_Storey_Benchmark_SA', 'Townhouse_Benchmark_SA',
       'Apartment_Benchmark_SA', 'Location'],
      dtype='object')

In [4]:
df = df.rename(columns={'Single_Family_Benchmark_SA': 'Single_Family', 
                        'One_Storey_Benchmark_SA': 'One_Storey',
                        'Two_Storey_Benchmark_SA': 'Two_Storey',
                        'Townhouse_Benchmark_SA' : 'Townhouse',
                        'Apartment_Benchmark_SA' : 'Apartment'
                        })

In [5]:
df.shape

(1989, 7)

In [6]:
%time
df.drop_duplicates(inplace = True)

CPU times: total: 0 ns
Wall time: 0 ns


In [7]:
df.shape

(1989, 7)

In [8]:
df=df.melt(id_vars = ['Date','Location'], var_name ='House_type', value_name='Benchmark_Price')
df.head

<bound method NDFrame.head of            Date                   Location     House_type  Benchmark_Price
0    2005-01-01           BRITISH_COLUMBIA  Single_Family         368300.0
1    2005-02-01           BRITISH_COLUMBIA  Single_Family         370200.0
2    2005-03-01           BRITISH_COLUMBIA  Single_Family         372700.0
3    2005-04-01           BRITISH_COLUMBIA  Single_Family         374800.0
4    2005-05-01           BRITISH_COLUMBIA  Single_Family         376900.0
...         ...                        ...            ...              ...
9940 2023-01-01  NEWFOUNDLAND_AND_LABRADOR      Apartment         231100.0
9941 2023-02-01  NEWFOUNDLAND_AND_LABRADOR      Apartment         228400.0
9942 2023-03-01  NEWFOUNDLAND_AND_LABRADOR      Apartment         223900.0
9943 2023-04-01  NEWFOUNDLAND_AND_LABRADOR      Apartment         222300.0
9944 2023-05-01  NEWFOUNDLAND_AND_LABRADOR      Apartment         226000.0

[9945 rows x 4 columns]>

In [9]:
df['Benchmark_Price'] = df['Benchmark_Price'].fillna(0)

In [10]:
import pyodbc

""" cnxn = pyodbc.connect(
    r'Driver=SQL Server;'
    r'Server=ADT230771;'
    r'Database=DWQueue;'
    r'Trusted_Connection=yes;'
    )
 """
cnxn = pyodbc.connect(
    r'Driver=SQL Server;'
    r'Server=DESKTOP-BN48D65\SQLEXPRESS;'
    r'Database=Housing;'
    r'Trusted_Connection=yes;'
    )
cursor = cnxn.cursor()
province = df['Location'].unique()
type = df['House_type'].unique()

codes =['BC','AB','SK','MB','ON','QC','NS','PE','NL']
values=list(zip(codes,province))

sql = "INSERT INTO PROVINCE (ProvinceCode,Name) VALUES (?,?)"
try:
    cursor.executemany(sql,values)
    cursor.commit()
except:
    pass  

sql = "INSERT INTO HouseType (Name) VALUES (?)"
try:
    cursor.executemany(sql,list(zip(type)))
    cursor.commit()
except:
    pass     
df.columns
for row in df.itertuples():

    cursor.execute('select TypeID from HouseType where Name = ?',row[3])
    type = cursor.fetchone()[0]
    cursor.execute('select ProvinceID from Province where Name = ?',row[2])
    prov = cursor.fetchone()[0]

    sql = "INSERT INTO HousePrices (Date,HouseType,Province,BenchMarkPrice) VALUES(?,?,?,?)"
    try:
        cursor.execute(sql,row[1],type,prov,row[4])
        cursor.commit()
    except:
        pass

cursor.close()
